In [1]:
import src
from src import oic_dwic
from src import ecif
from src import ms_oic
from src import utils

In [2]:
dwic = oic_dwic.InterAtomicContact(
    pathfiles="./test/",
    filename="dwic_fv.csv",
    ligand_format="mol2",
    amino_acid_classes=utils.amino_acid_classes_DWIC,
    cutoff=12.0,
    feature_type="DWIC",
    exp=2,
)

oic = oic_dwic.InterAtomicContact(
    pathfiles="./test/",
    filename="oic_fv.csv",
    ligand_format="mol2",
    amino_acid_classes=utils.amino_acid_classes_OIC,
    cutoff=12.0,
    feature_type="OIC",
    exp=None,
)

ecif = ecif.ECIF(
    pathfiles="./test/", filename="ecif_fv.csv", ligand_format="sdf", cutoff=6.0
)

ms_oic = ms_oic.MultiShellOIC(
    pathfiles="./test/", filename="msoic_fv.csv", ligand_format="mol2", n_shells=62
)

In [3]:
dwic.generate_features(n_jobs=-1)
oic.generate_features(n_jobs=-1)
ecif.generate_features(n_jobs=-1)
ms_oic.generate_features(n_jobs=-1)

Time: 0.0:0.0:7.15
Time: 0.0:0.0:2.10
Time: 0.0:0.0:3.04
Time: 0.0:0.0:8.47
